Lien de téléchargement des données: https://cvml.ista.ac.at/AwA2/        

13GB file : https://cvml.ista.ac.at/AwA2/AwA2-data.zip

In [2]:
import sys
import cv2 # Pour utiliser open_cv, il faut la version de python est 3.7
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms

np.random.seed(0)

In [48]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
JPEGIMAGES_FOLDER_PATH = "E:\\3710datas\\Animals_with_Attributes2\\JPEGImages\\"

In [49]:
# quick test
test = JPEGIMAGES_FOLDER_PATH+"fox\\fox_10001.jpg"
img = cv2.imread(test) 
print(img.shape) #ndarray
print(type(img))
cv2.imshow('Sample Image from AwA2 dataset',img)
cv2.waitKey(0)

(764, 918, 3)
<class 'numpy.ndarray'>


-1

In [50]:
labels_dirs = os.listdir(JPEGIMAGES_FOLDER_PATH)
print(labels_dirs)
len(labels_dirs) # 50 labels / subdirectories

['antelope', 'bat', 'beaver', 'blue+whale', 'bobcat', 'buffalo', 'chihuahua', 'chimpanzee', 'collie', 'cow', 'dalmatian', 'deer', 'dolphin', 'elephant', 'fox', 'german+shepherd', 'giant+panda', 'giraffe', 'gorilla', 'grizzly+bear', 'hamster', 'hippopotamus', 'horse', 'humpback+whale', 'killer+whale', 'leopard', 'lion', 'mole', 'moose', 'mouse', 'otter', 'ox', 'persian+cat', 'pig', 'polar+bear', 'rabbit', 'raccoon', 'rat', 'rhinoceros', 'seal', 'sheep', 'siamese+cat', 'skunk', 'spider+monkey', 'squirrel', 'tiger', 'walrus', 'weasel', 'wolf', 'zebra']


50

# Note : Some labels have a low number of images. 

## Possible solutions to explore : 
    Data augmentation : creating new training data by applying random transformations to existing images, such as rotating, cropping, or flipping them.

In [51]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH) -> tuple[dict,dict]: 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()
print(num_images_per_label)
print(proportions_images_per_label)

{'antelope': 1046, 'bat': 383, 'beaver': 193, 'blue+whale': 174, 'bobcat': 630, 'buffalo': 895, 'chihuahua': 567, 'chimpanzee': 728, 'collie': 1028, 'cow': 1338, 'dalmatian': 549, 'deer': 1344, 'dolphin': 946, 'elephant': 1038, 'fox': 664, 'german+shepherd': 1033, 'giant+panda': 874, 'giraffe': 1202, 'gorilla': 872, 'grizzly+bear': 852, 'hamster': 779, 'hippopotamus': 684, 'horse': 1645, 'humpback+whale': 709, 'killer+whale': 291, 'leopard': 720, 'lion': 1019, 'mole': 100, 'moose': 704, 'mouse': 185, 'otter': 758, 'ox': 728, 'persian+cat': 747, 'pig': 713, 'polar+bear': 868, 'rabbit': 1088, 'raccoon': 512, 'rat': 310, 'rhinoceros': 696, 'seal': 988, 'sheep': 1420, 'siamese+cat': 500, 'skunk': 188, 'spider+monkey': 291, 'squirrel': 1200, 'tiger': 877, 'walrus': 215, 'weasel': 272, 'wolf': 589, 'zebra': 1170}
{'antelope': 0.028, 'bat': 0.0103, 'beaver': 0.0052, 'blue+whale': 0.0047, 'bobcat': 0.0169, 'buffalo': 0.024, 'chihuahua': 0.0152, 'chimpanzee': 0.0195, 'collie': 0.0275, 'cow': 0.

In [53]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH): 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """
    
    labels_dirs:list = os.listdir(img_dir)
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Sucessfully created annotations.csv file.


In [58]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        label = self.img_labels.iloc[idx, 1]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [123]:
dataset = AWA2Dataset()

## TODO : Change transforms. Currently this is not useful.
dataset.transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    # transforms.ToTensor(), # Already a tensor as implemented in Dataset class with the reaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
                    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

# Testing. All good
# random_index = np.random.randint(0, len(dataset))
# image, label = dataset[1]
# print(label)
# print(image)
train_size =  int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,test_size])

tensor([[[152, 152, 154,  ..., 154, 151, 152],
         [154, 155, 153,  ..., 152, 152, 151],
         [157, 155, 155,  ..., 153, 151, 151],
         ...,
         [173, 170, 165,  ...,  91,  99, 102],
         [149, 173, 177,  ...,  91,  93,  96],
         [159, 172, 176,  ...,  91,  91,  92]],

        [[148, 146, 146,  ..., 164, 163, 162],
         [148, 149, 147,  ..., 162, 163, 161],
         [147, 149, 149,  ..., 163, 161, 163],
         ...,
         [143, 139, 132,  ..., 102, 102, 102],
         [121, 146, 149,  ...,  95,  97, 100],
         [130, 145, 148,  ...,  91,  93,  96]],

        [[137, 134, 133,  ..., 153, 149, 151],
         [136, 137, 135,  ..., 154, 149, 152],
         [140, 137, 135,  ..., 152, 151, 153],
         ...,
         [ 90,  92,  84,  ...,  98,  99, 102],
         [ 89, 101, 102,  ...,  94,  97, 101],
         [ 92, 101, 106,  ...,  89,  92,  95]]], dtype=torch.uint8) german+shepherd


In [142]:
image_train, label_train = train_dataset[1]
# print(image_train,label_train)
image_test, label_test = test_dataset[14]
# print(image_test,label_test)
print(len(train_dataset),len(test_dataset))

29856 7465


In [102]:
# Experiment with DataLoader. Everything works good
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True)
dataiter = iter(dataloader)
data = next(dataiter)

images, labels = data 
print(labels, images)

('moose', 'polar+bear', 'ox', 'rhinoceros') tensor([[[[ 93, 125,  88,  ...,  84,  45,  77],
          [ 95, 109,  96,  ..., 115,  76,  68],
          [ 94, 112, 154,  ..., 112,  63,  91],
          ...,
          [172, 171, 173,  ..., 167, 167, 161],
          [171, 175, 163,  ..., 167, 163, 163],
          [167, 164, 171,  ..., 166, 164, 166]],

         [[ 77, 109,  70,  ...,  69,  31,  56],
          [ 78,  91,  77,  ..., 100,  62,  46],
          [ 78,  94, 133,  ...,  97,  47,  70],
          ...,
          [170, 170, 171,  ..., 165, 165, 159],
          [168, 172, 162,  ..., 165, 161, 161],
          [165, 160, 170,  ..., 164, 162, 164]],

         [[ 68, 101,  59,  ...,  55,  24,  48],
          [ 67,  80,  64,  ...,  84,  50,  41],
          [ 64,  80, 118,  ...,  78,  32,  63],
          ...,
          [191, 192, 191,  ..., 179, 179, 173],
          [185, 191, 180,  ..., 179, 175, 175],
          [177, 173, 185,  ..., 178, 176, 178]]],


        [[[182, 179, 183,  ...,  79,  5

In [108]:
# Training loop example
num_epochs = 2 
batch_size = 4
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/batch_size)
print(total_samples, n_iterations)

dataloader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs) : 
    # loop over trainloader 
    for i, (inputs, labels) in enumerate(dataloader) : 
        
        # Do forward and backward pass, update the weights 
        if(i+1) % 5 == 0 :
            print(f'epoch {epoch+1} / {num_epochs}, step, {i+1}/{n_iterations}, inputs {inputs}, label {labels}')

        if i==20 : 
            print('Completed')
            break


37321 9331
epoch 1 / 2, step, 5/9331, inputs tensor([[[[ 45,  44,  45,  ...,   1,   0,   0],
          [ 41,  42,  54,  ...,   1,   0,   0],
          [ 46,  50,  50,  ...,   0,   0,   0],
          ...,
          [  0,   0,   0,  ...,   0,   0,   2],
          [  0,   0,   0,  ...,   0,   0,   2],
          [  0,   0,   0,  ...,   0,   0,   1]],

         [[ 96,  96,  96,  ...,  60,  65,  64],
          [ 95,  95, 104,  ...,  64,  62,  65],
          [ 97,  99, 100,  ...,  91,  87,  87],
          ...,
          [ 74,  73,  72,  ...,  50,  47,  50],
          [ 73,  73,  74,  ...,  49,  49,  50],
          [ 75,  74,  73,  ...,  50,  50,  49]],

         [[127, 125, 123,  ...,  86,  91,  91],
          [125, 125, 131,  ...,  94,  93,  94],
          [126, 129, 127,  ..., 118, 115, 113],
          ...,
          [103, 105, 104,  ...,  77,  73,  73],
          [104, 105, 105,  ...,  76,  74,  73],
          [106, 105, 105,  ...,  75,  74,  72]]],


        [[[130, 131, 131,  ...,  60,  

In [145]:
import tensorflow as tf

TypeError: Couldn't build proto file into descriptor pool: duplicate symbol 'tensorflow.CoordinatedJob'